In [1]:
import json
import random


data = json.load(open('../dataset/Jurassic_Ultra/gender/raw.json'))
anno_data = []
for k, v in data.items():
    
    responses = list(zip(range(len(v['other_attributes'])), v['other_attributes']))
    chosen_triples = []
    for _ in range(10):
        r1, r2, r3 = random.sample(responses, k=3)
        if (r1[0], r2[0], r3[0]) not in chosen_triples and (r1[0], r3[0], r2[0]) not in chosen_triples:
            chosen_triples.append((r1[0], r2[0], r3[0]))
            chosen_triples.append((r1[0], r3[0], r2[0]))
            ex = {}
            ex['group_id'] = k
            ex['base_question'] = v['base_question']
            ex['r1'] = r1[1]
            ex['r2'] = r2[1]
            ex['r3'] = r3[1]
            anno_data.append(ex)
print(f'Total data size: {len(anno_data)}')

Total data size: 419


In [3]:
n_annotator = 5
n_per_annotator = len(anno_data) // n_annotator


for i in range(n_annotator):
    data = anno_data[i*(n_per_annotator): (i+1)*n_per_annotator]
    print(len(data))
    filename = 'anno_data_%02d.json' % (i+1)
    json.dump(data, open(filename, 'w'), indent=2)


83
83
83
83
83
